### Clustering

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import re
import os
import ast
import torch
import json
import random

In [16]:
# articles
with open('../../data/usernames_with_titles_updated.json', 'r') as file:
    articles = json.load(file)
    
print(articles.keys())
print(dict(list(articles.items())[:10]))
print(len(articles.keys()))


dict_keys(['!---slappdash---!', '%D0%90', "'sed", '(.Y.)', '(:Julien:)', '(aeropagitica)', '*Kat*', '*drew', '-Midorihana-', '-Ril-', '-asx-', '-jha-', '.:Ajvol:.', '.V.', '.snoopy.', '068152', '069952497a', '0pen$0urce', '10014derek', '1234r00t', '123Hedgehog456', '125.162.150.88', '131.211.210.14', '131.211.210.157', '156.34.142.110', '172', '17Drew', '194.6.242.2', '1948remembered', '1966batfan', '198', '198.110.249.200', '1ForTheMoney', '1of3', '2', '2004-12-29T22:45Z', '202.58.85.8', '203.125.28.98', '203.213.7.131', '205.157.110.11', '207.6.58.42', '211.30.169.132', '216.153.214.94', '21655', '217.33.207.195', '23prootie', '23skidoo', '24.15.158.90', '24.18.171.99', '24.46.201.42', '24.55.228.56', '24ip', '28bytes', '2D', '2help', '2over0', '32X', '33451', '3centsoap', '3leopard', '43?9enter', '4kinnel', '4twenty42o', '4wajzkd02', '578', '5faizan', '64.12.116.133', '68.190.178.87', '69.121.245.182', '69.57.154.67', '6SJ7', '7', '71.106.28.156', '71.112.134.138', '7107delicious', 

In [20]:
# categories
with open('../../data/user_categories.json', 'r') as file:
    categories_old = json.load(file)
    
print(categories_old.keys())
print(dict(list(categories_old.items())[:10]))
print(len(categories_old.keys()))

with open('../../data/user_categories_new.json', 'r') as file:
    categories_new = json.load(file)
    
print(categories_new.keys())
print(dict(list(categories_new.items())[:10]))
print(len(categories_new.keys()))

categories = categories_old | categories_new
print(categories.keys())
print(dict(list(categories.items())[:10]))
print(len(categories.keys()))


dict_keys(['!---slappdash---!', '%D0%90', "'sed", '(.Y.)', '(aeropagitica)', '*Kat*', '*drew', '-Midorihana-', '-Ril-', '123Hedgehog456', '125.162.150.88', '131.211.210.14', '131.211.210.157', '156.34.142.110', '172', '17Drew', '194.6.242.2', '1948remembered', '1966batfan', '198', '198.110.249.200', '1ForTheMoney', '1of3', '2', '202.58.85.8', '203.125.28.98', '203.213.7.131', '205.157.110.11', '207.6.58.42', '211.30.169.132', '216.153.214.94', '21655', '217.33.207.195', '23prootie', '23skidoo', '24.15.158.90', '24.18.171.99', '24.46.201.42', '24.55.228.56', '24ip', '28bytes', '2D', '2help', '2over0', '32X', '33451', '3centsoap', '3leopard', '4kinnel', '4twenty42o', '4wajzkd02', '578', '5faizan', '64.12.116.133', '68.190.178.87', '69.121.245.182', '69.57.154.67', '6SJ7', '7', '71.106.28.156', '71.112.134.138', '7107delicious', '71Demon', '74.178.230.17', '75.62.7.22', '76', '78.26', '7OA', '80.176.190.96', '82', '83d40m', '8bitJake', '90', '95j', '99of9', '9cds', 'A', 'A.Garnet', 'A.Sav

In [36]:
admin_score=pd.read_csv('../../data/scores.csv')
admin_score_crop=admin_score.drop_duplicates()
print(len(admin_score_crop['username'].unique()))
print(admin_score_crop)


8026
          username  total_score
0       (:Julien:)          592
1     -Midorihana-          693
2            -jha-          108
3       069952497a          864
4         1234r00t          649
...            ...          ...
8293       wpktsfs          544
8294         ybbor          755
8295      Þjóðólfr          533
8296         Вasil          629
8297             石          420

[8029 rows x 2 columns]


In [59]:
#merge in dataframe
fullData=admin_score_crop.copy()

data = []
for username, items in categories.items():
    rows={'username': username}
    rows.update({f"categ{i+1}":cat for i, cat in enumerate(items)}) 
    data.append(rows)
categ_dataframe=pd.DataFrame(data)
categ_dataframe=categ_dataframe.drop(columns=['categ5'])
print(categ_dataframe.head())
data = []
for username, items in articles.items():
    rows={'username': username}
    rows.update({f"articles{i+1}":cat for i, cat in enumerate(items)}) 
    data.append(rows)
articles_dataframe=pd.DataFrame(data)
#articles_dataframe=articles_dataframe.drop(columns=['categ5'])
articles_dataframe



            username         categ1         categ2     categ3      categ4
0  !---slappdash---!  Entertainment          Music   Business      Sports
1             %D0%90      Education           Film  Geography       Media
2               'sed      Animation         Comedy      Media  Television
3              (.Y.)          Music          Lists     People      Sports
4     (aeropagitica)          Media  Entertainment    Culture     Science


,username,articles1,articles2,articles3,articles4,articles5,articles6,articles7,articles8,articles9,articles10
0,!---slappdash---!,"[Rupture (band), 30]","[The Cobra Group, 22]","[Trasharama A-Go-Go, 19]","[Blood Duster, 12]","[Propagandhi, 3]","[Australian hardcore, 3]","[Brutal Truth, 2]","[Dropdead, 1]","[Wacken Open Air, 1]","[Pyramid scheme, 1]"
1,%D0%90,"[List of Sundance Film Festival award winners,...",[List of films at the 2008 Sundance Film Festi...,"[Weber State University, 18]","[Ogden, Utah, 17]","[2008 Sundance Film Festival, 14]","[List of MyNetworkTV affiliates, 13]","[KTVX, 12]",[List of films at the 2007 Sundance Film Festi...,[List of ABC television affiliates (by U.S. st...,"[2006 Sundance Film Festival, 11]"
2,'sed,"[The Mansion Family, 1]","[Bloody Mary (South Park), 1]","[Cartoon Wars Part I, 1]","[Cartoon Wars Part II, 1]","[The Eight I'd Really Rather You Didn'ts, 1]",NaN,NaN,NaN,NaN,NaN
3,(.Y.),"[Slayer, 2]","[List of drummers, 1]","[List of bass guitarists, 1]","[Videogroove, 1]","[Blue Water High, 1]",NaN,NaN,NaN,NaN,NaN
4,(:Julien:),"[2012 FIBA Europe Under-18 Championship, 11]","[2013 FIBA Europe Under-20 Championship, 9]",[Party of Communists of the Republic of Moldov...,[List of names of European cities in different...,"[2011 FIBA Europe Under-16 Championship, 6]","[List of twin towns and sister cities in Asia, 5]","[Roch Marc Christian Kaboré, 5]","[Zeev Sternhell, 5]","[Third Zimmerwald Conference, 4]","[Nikolay Zabolotsky, 4]"
...,...,...,...,...,...,...,...,...,...,...,...
7810,Waldyrious,[List of text-based massively multiplayer onli...,"[Cape Verde, 116]","[List of films featuring extraterrestrials, 69]","[Grading in education, 66]","[Submerged floating tunnel, 46]","[ECTS grading scale, 39]","[History of Cape Verde, 38]","[Etherpad, 35]","[Collaborative real-time editor, 31]","[Cape Verdean Creole, 30]"
7811,Warren,"[Windows Vista, 989]","[Windows 7, 264]","[Features new to Windows Vista, 237]","[Windows XP, 198]","[Development of Windows Vista, 162]","[Mac OS X Leopard, 147]","[MacOS, 138]","[Internet Explorer, 138]","[Microsoft Windows, 128]","[Features new to Windows 7, 121]"
7812,Wes!,"[Huntington Beach High School, 36]","[Christopher Columbus, 12]","[Online game, 11]","[Lisa Tucker (singer), 10]","[Indian Idol, 10]","[Suburban Legends (EP), 9]","[Shit, 9]","[Forrest Kline, 8]","[Memorial High School (Hedwig Village, Texas), 7]","[Nelson Mandela, 7]"
7813,Emma Caroline,"[Young's modulus, 7]","[WHRE, 7]","[WOTV, 7]","[Breakfast at Twilight, 6]","[Jeffrey Rosenberg, 6]","[2012 Pacific hurricane season, 6]","[Encoding specificity principle, 6]","[Jackie Mitchell, 5]","[S. E. Hinton, 5]","[1993–94 French Division 1, 5]"


NameError: name 'cat_dataframe' is not defined